In [3]:
# Basic Imports
import numpy as np

In [2]:
# Reference: https://ieeexplore.ieee.org/document/5980409
# Convention: WRB = WRC * CRB (means body is first rotated via pitch and roll, then rotated via yaw into world frame)

# Angular velocity of body frame in world frame where p q r are the angular velocities in the body frame and xb yb zb are the body frame axes (3D unit vectors).
# p, q, r is roll, pitch, yaw respectively.
def body_angular_velocity_to_world(p, q, r, xb, yb, zb):
    return np.array([
        p * xb + q * yb + r * zb,
        p * yb + q * zb - r * xb,
        p * zb - q * xb + r * yb
    ]).T



In [1]:
# Each rotor has an angular speed wi and produces a force Fi and a moment Mi.
# Force is in the direction of the rotor axis and is perpendicular to the plane of rotation, while moment is in the plane of rotation.

# The force Fi is given by:
# Fi = Fk * wi^2
# where Fk is a constant.

# The moment Mi is given by:
# Mi = Mk * wi^2
# where Mk is a constant.

def rotor_force(wi, Fk):
    return Fk * wi**2
def rotor_moment(wi, Mk):
    return Mk * wi**2

# Given motor dynamics are comparatively fast, we can assume they are instantaneous. This means we can turn the control inputs into forces and moments.
# u1 is the net body force, u2 is the roll moment, u3 is the pitch moment, and u4 is the yaw moment.
# L is the distance from the center of rotation to the vehicle center.

# The net body force is given by:
    # u1 = F1 + F2 + F3 + F4
# The roll moment is given by:
    # u2 = F2*L - F4*L = L*(F2 - F4)
# The pitch moment is given by:
    # u3 = F3*L - F1*L = L*(F3 - F1)
# The yaw moment is given by:
    # u4 = M1 - M2 + M3 - M4

# Therefore, as a matrix, separating out the angular velocities, we can write:
def rotor_forces_moments(w1, w2, w3, w4, L, Fk, Mk):
    motors = np.array([
        [Fk, Fk, Fk, Fk],
        [0, Fk*L, 0, -Fk*L],
        [-Fk*L, 0, Fk*L, 0],
        [Mk, -Mk, Mk, -Mk]
    ])
    speeds = np.array([w1**2, 
                       w2**2, 
                       w3**2, 
                       w4**2])
    forces_moments = motors @ speeds
    return forces_moments

In [ ]:
# The position of the quadrotor in world frame is given by "r".
# The acceleration treating the quad as a point mass in world frame is given by:
# m * rddot = u1 - mg